<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/ForwardTacotron_Espa%C3%B1ol_Entrenamiento(beta7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno de entrenamiento de `Forward Tacotron` (En español).
Este cuaderno ha sido desarrollado por [rmcpantoja](https://github.com/rmcpantoja).

Colaborador y corrector del cuaderno: [Xx_Nessu_XX](https://fakeyou.com/profile/Xx_Nessu_xX)
## Créditos:

* Repositorio de [as-ideas/ForwardTacotron](https://github.com/as-ideas/ForwardTacotron).


### <ins>¡IMPORTANTE!</ins>:

* Por ahora, este cuaderno no está óptimo para entrenar con conjuntos de datos pequeños. Estoy planeando hacer un modelo preentrenado con un dataset 24 horas.


*Última actualización: 23/02/2023*

In [ ]:
#@markdown ### Comprobar la GPU asignada.
#@markdown ---
#@markdown Necesitas una `Tesla T4` como mínimo. Si tienes una GPU como `K80`, ve a la barra de menús y selecciona Entorno de ejecución > Desconectarse y eliminar entorno de ejecución.
#@markdown * También puedes ejecutar este cuaderno sin GPU (No recomendado) desactivando la aceleración de hardware en la configuración del cuaderno.
!nvidia-smi -L

In [ ]:
#@markdown ### Montar Google drive.
#@markdown ---
#@markdown Esto es muy importante para almacenar los puntos de control y los conjuntos de datos procesados con los que Forward Tacotron podrá trabajar. Sin embargo, algunas notas importantes:
#@markdown * Es importante que verifiques tu espacio de almacenamiento en [Drive](http://drive.google.com/). De acuerdo al tamaño del dataset, necesitas calcular una mayor cantidad de espacio disponible.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ## Iniciar el proceso de instalación.
#@markdown ---
#@markdown Esto instalará el sintetizador y otras dependencias importantes.
#@markdown #### <font color=orange>¿Quieres utilizar la versión 3.2 de ForwardTacotron?

#@markdown La versión 3.2 es la última que sólo contiene soporte para un único hablante.
only_singlespeaker_version = True #@param {type:"boolean"}
#@markdown ---
%cd /content
import os
from os.path import exists
if (not os.path.exists("/content/ForwardTacotron")):
  print("clonando repositorio...")
  !git clone https://github.com/as-ideas/ForwardTacotron
else:
  print("El repositorio de trabajo ya existe. Saltando...")
# pip:
!pip install numba librosa pyworld phonemizer webrtcvad PyYAML dataclasses soundfile scipy tensorboard matplotlib unidecode inflect resemblyzer==0.1.1-dev pandas
!pip install --upgrade gdown
#!pip install git+https://github.com/wkentaro/gdown.git
%cd /content/ForwardTacotron
if only_singlespeaker_version.
  !git checkout 632b453c6cb6d15dfe4dd168cd60c60e56731829
!rm -r .git/
#apt:
!apt install espeak-ng
!wget https://github.com/mikefarah/yq/releases/download/v4.29.2/yq_linux_amd64.tar.gz
!tar -xvf yq_linux_amd64.tar.gz
!mv /content/ForwardTacotron/yq_linux_amd64 /content/ForwardTacotron/yq
#!bash install-man-page.sh
print("Listo")

# Aplicar parches.

## **Ejecutar después de la celda de configuración.**

In [ ]:
name = "test"
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [ ]:
%%writetemplate /content/ForwardTacotron/utils/paths.py
import os
from pathlib import Path


class Paths:
    """Manages and configures the paths used by WaveRNN, Tacotron, and the data."""
    def __init__(self, data_path, tts_id):

        # directories
        self.base = Path(__file__).parent.parent.expanduser().resolve()

        # Data Paths
        self.data = Path(data_path).expanduser().resolve()
        self.quant = self.data/'quant'
        self.mel = self.data/'mel'
        self.gta = self.data/'gta'
        self.att_pred = self.data/'att_pred'
        self.alg = self.data/'alg'
        self.speaker_emb = self.data/'speaker_emb'
        self.mean_speaker_emb = self.data/'mean_speaker_emb'
        self.raw_pitch = self.data/'raw_pitch'
        self.phon_pitch = self.data/'phon_pitch'
        self.phon_energy = self.data/'phon_energy'
        self.model_output = self.base / 'model_output'
        self.save_dir = Path("{save_dir}").expanduser().resolve()
        self.taco_checkpoints = self.save_dir/'checkpoints/{tts_id}.tacotron'
        self.taco_log = self.taco_checkpoints / 'logs'
        self.forward_checkpoints = self.save_dir/'checkpoints/{tts_id}.forward'
        self.forward_log = self.forward_checkpoints/'logs'

        # pickle objects
        self.train_dataset = self.data / 'train_dataset.pkl'
        self.val_dataset = self.data / 'val_dataset.pkl'
        self.text_dict = self.data / 'text_dict.pkl'
        self.speaker_dict = self.data / 'speaker_dict.pkl'
        self.att_score_dict = self.data / 'att_score_dict.pkl'

        self.create_paths()

    def create_paths(self):
        os.makedirs(self.data, exist_ok=True)
        os.makedirs(self.quant, exist_ok=True)
        os.makedirs(self.mel, exist_ok=True)
        os.makedirs(self.gta, exist_ok=True)
        os.makedirs(self.alg, exist_ok=True)
        os.makedirs(self.speaker_emb, exist_ok=True)
        os.makedirs(self.mean_speaker_emb, exist_ok=True)
        os.makedirs(self.att_pred, exist_ok=True)
        os.makedirs(self.raw_pitch, exist_ok=True)
        os.makedirs(self.phon_pitch, exist_ok=True)
        os.makedirs(self.phon_energy, exist_ok=True)
        os.makedirs(self.taco_checkpoints, exist_ok=True)
        os.makedirs(self.forward_checkpoints, exist_ok=True)

    def get_tts_named_weights(self, name):
        """Gets the path for the weights in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_weights.pyt'

    def get_tts_named_optim(self, name):
        """Gets the path for the optimizer state in a named tts checkpoint."""
        return self.taco_checkpoints / f'{name}_optim.pyt'

    def get_voc_named_weights(self, name):
        """Gets the path for the weights in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_weights.pyt'

    def get_voc_named_optim(self, name):
        """Gets the path for the optimizer state in a named voc checkpoint."""
        return self.voc_checkpoints/f'{name}_optim.pyt'




# Preparación del proyecto.

In [ ]:
%cd /content/ForwardTacotron
#@markdown ### Configuración.

#@markdown Estas son algunas configuraciones con las que podremos modificar ajustes relacionados a datos y entrenamiento.

#@markdown ---
#@markdown #### Elige la variante de modelo a usar:
import os
model_type = "Un solo hablante" #@param ["Un solo hablante", "Varios hablantes"]
if model_type == "Un solo hablante":
  config_path = "configs/singlespeaker.yaml"
elif model_type == "Varios hablantes":
  config_path = "configs/multispeaker.yaml"
elif only_singlespeaker_version:
  config_path = "config.yaml"
else:
  raise Exception("Tipo de modelo no soportado. Actualmente, puedes elegir entre un solo hablante, varios o no tomarlo en cuenta si usas la versión 3.2.")
#@markdown ---
#@markdown #### Nombre deseado para el modelo:
tts_model_id = "EjemploTTS" #@param {type:"string"}
tts_id = tts_model_id
!./yq -i '.tts_model_id = "{tts_model_id}"' "{config_path}"
#@markdown ---
# waveRNN vocoder removed from last version
#@markdown #### Por favor, elige el tipo de modelo que se va a entrenar con este conjunto de datos:

#@markdown La opción `multi_forward_tacotron` está soportada solamente para modelos de varios hablantes.
tts_model = "forward_tacotron" #@param ["forward_tacotron", "multi_forward_tacotron", "fast_pitch"]
if tts_model == "multi_forward_tacotron" and model_type == "Un solo hablante":
  raise Exception("El modelo multi_forward_tacotron solo está soportado en modelos de varios hablantes.")
elif tts_model == "multi_forward_tacotron" and only_singlespeaker_version:
  raise Exception("El modelo multi_forward_tacotron no está soportado en esta versión.")
!./yq -i '.tts_model = "{tts_model}"' "{config_path}"
#@markdown ---
#@markdown #### ¿Continuar un entrenamiento?
continue_training = False #@param {type:"boolean"}
#@markdown Puedes establecer la ubicación del conjunto de datos preprocesado que se guardó a tu Drive
preprocess_path = "/content/drive/MyDrive/ForwardTacotron/EjemploTTS/dataset_preprocessed.zip" #@param {type:"string"}
if continue_training:
  !unzip $preprocess_path -d /content/FforwardTacotron
#@markdown ---
#@markdown #### ¿Guardar puntos de control y preprocesamiento en una ubicación personalizada?
custom_save_dir = True #@param {type:"boolean"}

#@markdown Si está activado, ¿dónde deseas guardarlo?
save_dir = "/content/drive/MyDrive/ForwardTacotron/EjemploTTS" #@param {type:"string"}
if custom_save_dir:
  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
else:
  print("¡Advertencia! Los cambios no se guardarán, solo en la carpeta local del proyecto.")
  save_dir = "/content/ForwardTacotron"
#@markdown ---
#@markdown #### Elige la frecuencia de muestreo: (Opcional)
sample_rate = "22050" #@param ["16000", "22050", "24000", "32000", "44100", "48000"] {allow-input: true}
!./yq -i '.dsp.sample_rate = {sample_rate}' "{config_path}"
!./yq -i '.dsp.vad_sample_rate = {sample_rate}' "{config_path}"
#@markdown ---
#@markdown #### Elige el formato del archivo de transcripciones:

#@markdown El formato `ljspeech` es el único que se usa para modelos de un solo hablante.

metafile_format = "ljspeech" #@param ["ljspeech", "ljspeech_multi", "pandas", "vctk"]
if metafile_format == "ljspeech" and model_type == "Varios hablantes":
  raise Exception("El formato ljspeech es compatible solamente con los modelos de un solo hablante.")
elif not metafile_format == "ljspeech" and only_singlespeaker_version:
  raise Exception("El formato ljspeech es el único soportado en la versión 3.2")
else:
  !./yq -i '.preprocessing.metafile_format = "{metafile_format}"' "{config_path}"
#@markdown ---
#@markdown #### Número de validaciones (Puedes ajustarlo según el tamaño del conjunto de datos).
n_val = 100 #@param {type:"integer"}
!./yq -i '.preprocessing.n_val = {n_val}' "{config_path}"
#@markdown ---
#@markdown #### Elige la bariación de idioma en el que tiene este conjunto de datos:
#@markdown * Tengamos en cuenta que "`es`" equivale a español de españa (recomendado) y "`es-419`" equivale al español de latinoamérica.
language = 'es' #@param ["es", "es-419"]
!./yq -i '.preprocessing.language = "{language}"' "{config_path}"
#@markdown ---
# set no cleaners:
cleaner_name = 'no_cleaners'
!./yq -i '.preprocessing.cleaner_name = "{cleaner_name}"' "{config_path}"
# reduce workers in dur extraction:
!./yq -i '.duration_extraction.num_workers = "2"' "{config_path}"
#@markdown #### Intervalo de pasos para generar señales de entrenamiento del modelo:
#@markdown Aquí podremos configurar cada cuántos pasos se generarán figuras, imágenes, visuales y audio, es decir, el progreso del entrenamiento en que se podrá ver en tensorboard (Más adelante).
#@markdown * Nota: esta configuración aplicará en todos los modelos: Tacotron, Forward_tacotron, multi_forward_tacotron (Si se entrena con varios hablantes), y FastPitch.
plot_every = 5000 #@param {type:"integer"}
!./yq -i '.tacotron.training.plot_every = {plot_every}' "{config_path}"
if model_type == "Varios hablantes":
  !./yq -i '.multi_forward_tacotron.training.plot_every = {plot_every}' "{config_path}"
else:
  !./yq -i '.forward_tacotron.training.plot_every = {plot_every}' "{config_path}"
!./yq -i '.fast_pitch.training.plot_every = {plot_every}' "{config_path}"
#@markdown ---
# phoneme singlespeaker:
!./yq -i '.preprocessing.use_phonemes = "True"' "{config_path}"
# attention:
if model_type == "Varios hablantes":
  !./yq -i '.multi_forward_tacotron.training.filter_attention = True' "{config_path}"
  !./yq -i '.multi_forward_tacotron.training.min_attention_sharpness = 0.25' "{config_path}"
  !./yq -i '.multi_forward_tacotron.training.min_attention_alignment = 0.5' "{config_path}"
else:
  !./yq -i '.forward_tacotron.training.filter_attention = True' "{config_path}"
  !./yq -i '.forward_tacotron.training.min_attention_sharpness = 0.25' "{config_path}"
  !./yq -i '.forward_tacotron.training.min_attention_alignment = 0.5' "{config_path}"

## Trabajando con el conjunto de datos.

**Puedes saltarte estas celdas si ya pre-procesaste un dataset por primera vez y quieres entrenarlo en el último punto de control que se haya guardado. De lo contrario, expande esta sección y lee las instrucciones de cada celda.**

In [ ]:
import zipfile
import os
import os.path
#@markdown ### Procesamiento del conjuntos de datos.
#@markdown ---
#@markdown * Nota: si vas a preprocesar conjuntos de datos de mayor tamaño, se recomienda tener más espacio disponible en drive.
#@markdown ---
#@markdown #### Ruta de los audios. Ellos deberán almacenarse en un archivo .zip:
wavs_path = "/content/drive/MyDrive/wavs.zip" #@param {type:"string"}
#@markdown ---
#@markdown #### Ruta de transcripción: (Por defecto metadata.csv)
list_path = "/content/drive/MyDrive/list.csv" #@param {type:"string"}
list_filename = os.path.basename(list_path).split('/')[-1]
#@markdown ---
%cd /content
!mkdir dataset
%cd dataset
!mkdir wavs
if zipfile.is_zipfile(wavs_path):
  !unzip -j "$wavs_path" -d /content/dataset/wavs
else:
  print("Aviso: la ruta de audios no es un archivo comprimido.")
if list_path.endswith('.txt'):
  raise Exception("El formato de la transcripción deberá estar en formato csv")

if not os.path.exists(list_path):
  raise Exception("Error: el archivo de transcripción no existe, inténtelo de nuevo por favor.")
else:
  !cp $list_path /content/dataset
%cd /content/ForwardTacotron
print("Ejecutando procesamiento...")
if only_singlespeaker_version:
  !python preprocess.py --path /content/dataset
else:
  !python preprocess.py --path /content/dataset --config "{config_path}" --metafile "{list_filename}"
if custom_save_dir:
  print("Respaldando preprocesamiento...")
  !zip -r "$save_dir/dataset_preprocessed.zip" configs data

### <font color='red'>¡Precaución!</font> Debes ejecutar esta celda si tienes un conjunto de datos en tu Forward Tacotron y quieres entrenar otro. Los contenidos se borrarán.

In [ ]:
#@markdown ### <font color='red'>Borrar el conjunto de datos actual (si existe)
#@markdown ---
#@markdown Debido a que los datasets se encuentran en la carpeta de trabajo, es posible que necesites entrenar otro datasetp. Si es así, ejecuta esta celda para hacerlo.
# conjunto de datos
!rm -rf /content/ForwardTacotron/dataset
# preprocesado:
!rm -rf /content/ForwardTacotron/data/*

# ¡A entrenar!
Esta serie de pasos requerirán de tiempo para conseguir un entrenamiento estable y tras horas, y a veces algunos días, obtener los resultados finales. Por favor, sugiero leer atentamente las indicaciones de cada una de las celdas.

***¡Modelo preentrenado español pronto!***

In [ ]:
#@markdown ### Ejecutar la extensión Tensorboard.
#@markdown ---
#@markdown El tensorboard sirve para visualizar el proceso de entrenamiento del modelo. Ten en cuenta que si quieres visualizar esto, puedes ir a las pestañas **audio**, **image** o **scalars**.
%load_ext tensorboard
%tensorboard --logdir "{data_path}/checkpoints"
import tensorflow as tf
import datetime

In [ ]:
#@markdown ### Entrenamiento 1: Tacotron.
#@markdown ---
#@markdown Un punto muy a tomar en cuenta entre Tacotron y Forward Tacotron es la división del entrenamiento.
#@markdown * El modelo se entrenará entre un total de 40k pasos. De forma predeterminada, los respaldos se guardan cada 10k pasos, por lo que deberíamos preocuparnos por el almacenamiento. ___(En cambio, puedes borrar los respaldos antiguos. Igualmente, el que realmente importa y se usa es el latest_model que se guarda más a menudo).___
#@markdown * Asimismo, este entrenamiento cumple un cronograma el cual se aplicarán parámetros diferentes.
#@markdown
#@markdown Sin más, ¡a entrenar!.
if only_singlespeaker_version:
  !python train_tacotron.py
else:
  !python train_tacotron.py --config "{config_path}"
# include att score, pitch, att, aligments and more:
print("Pero antes, respaldando el trabajo que se acaba de hacer...")
!zip –u "$save_dir/dataset_preprocessed.zip" configs data
print("¡Listo!")

In [ ]:
#@markdown ### Entrenamiento 2: ForwardTacotron.
#@markdown ---
#@markdown Esto entrenará el modelo final para Forward Tacotron, tomando en cuenta el trabajo realizado anteriormente.
#@markdown * Asimismo, cumplirá un cronograma. Por defecto, se entrena hasta 300k pasos, pero puede funcionar con menos.
#@markdown * Recuerda que se tomará en cuenta la atención basándose en el modelo Tacotron. Si se está entrenando con pocos archivos debido a la mala atención (Podemos darnos cuenta de esto durante el entrenamiento), hay un problema en el conjunto de datos. Así que por favor, procura revisarlo, arreglar lo que sea necesario o aumentar más datos.
#@markdown * Puedes ajustar el tamaño del lote (Batch) si no tienes memoria durante la sesión aquí.
batch_size = 24 #@param {type:"integer"}
!./yq -i '.forward_tacotron.training.schedule[0] = 5e-5,  150_000,  {batch_size}' "{config_path}"
!./yq -i '.forward_tacotron.training.schedule[1] = 1e-5,  300_000,  {batch_size}' "{config_path}"
#@markdown ---
if only_singlespeaker_version:
  !python train_forward.py
else:
  !python train_forward.py --config "{config_path}"


# ¿Has terminado de entrenar por hoy?
Prueba el modelo en el cuaderno de síntesis pulsando [aquí.](https://colab.research.google.com/drive/1Y8tTrw3OSp8eJJryRgpbLkEdxnzwI5Cv)